#[1] Mount Google Drive

In [ ]:
from google.colab import drive
import os
import sys
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#[2] Setting up Environment and Install Requirements

In [ ]:
%%capture
!pip install pandas==1.3.2
!pip install transformers
!pip install -r sources/ChinaFilm/requirements.txt
#!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [1]:
# Set Path
path_wd = '/content/drive/MyDrive/Github/Content'
os.chdir(path_wd)

# Import External Libs
import pandas as pd
import numpy as np
import datetime as dt
import re
import random

from IPython.display import display, HTML
import torch
from transformers import AdamW, BartForConditionalGeneration

# Import External Libs
import pandas as pd
import numpy as np
import datetime as dt
import re
import random
from selenium import webdriver

import torch
from transformers import AdamW, BartForConditionalGeneration

import jieba.analyse
tfidf = jieba.analyse.extract_tags
textrank = jieba.analyse.textrank

import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.font_manager as fm

path_fonts = '/content/drive/MyDrive/Github/Article/fonts'
fontprop = fm.FontProperties(fname=path_fonts, size= 15)

font_dirs = [path_fonts, ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
#font_list = fm.createFontList(font_files)
for font in font_files:
  fm.fontManager.addfont(font)

plt.rcParams['figure.figsize'] = [15, 9]
mp.rcParams['font.family'] = ['Microsoft YaHei']

%matplotlib inline

NameError: ignored

In [ ]:
# Define Path
path_records = '/content/drive/MyDrive/Github/Article/records'

# [3] Load Release and Registration Records

## Maoyan 猫眼

In [ ]:
# Update Maoyan Releases - Run only if necessary
path_maoyan = '/content/drive/MyDrive/Github/Content/sources/maoyan'
os.chdir(path_maoyan)
!python search_curr_releases.py

search result saved to: searches/search__20210928_2209.json  with 22 records
file saved to: records_maoyan.json with a total of  22  records.


In [ ]:
path_maoyan_releases = path_maoyan + '/records/' + 'records_maoyan.json'
df_maoyan = pd.read_json(path_maoyan_releases)
df_maoyan = df_maoyan.drop_duplicates('fid')

In [ ]:
df_maoyan

## ChinaFilm 电影局

In [ ]:
# Update Release Records
path_reg = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm'
os.chdir(path_reg)
#!python Update_Release.py

In [ ]:
# Update Reg Records
path_reg = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm'
os.chdir(path_reg)
#!python Update_Registration.py

In [ ]:
# Import Releases
path_releases = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm/records/contents_of_releases.json'
df_releases = pd.read_json(path_releases)

# Import Reg
path_regs = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm/records/contents_of_registrations.json'
df_reg = pd.read_json(path_regs, orient='record')
df_reg['匹配片名'] = df_reg['片名']

# Get Latest Issue
str_date_curr_release = df_releases.sort_values(by='公示日期', ascending=False).iloc[0]['公示日期']
dt_curr_release = dt.datetime.strptime(str_date_curr_release, '%Y-%m-%d')
df_curr_releases = df_releases[df_releases['公示日期']==str_date_curr_release]
df_curr_releases = df_curr_releases[['电影类别', '编码', '公映证号', '片名', '第一出品单位']].copy()

In [ ]:
# Merge Maoyan info into df_curr_releases
df_curr_releases = df_curr_releases.merge(df_maoyan, on='片名', how='left')
df_curr_releases

In [ ]:
# Export current release to csv for Power Automate processing
df_curr_releases.to_csv(
    path_records + '/df_curr_releases_{}.csv'.format(str_date_curr_release),
    encoding='utf-8-sig'
)

In [ ]:
str_date_curr_release

'2021-09-23'

In [ ]:
df_curr_releases.columns

Index(['电影类别', '编码', '公映证号', '片名', '第一出品单位', 'fid', '主演', '上映日期', '想看'], dtype='object')

In [ ]:
df_reg.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '电影类别', '匹配片名'],
      dtype='object')

#[4] Interactively Find matches

In [ ]:
# Load Libraries Local
path_Baike = '/content/drive/MyDrive/Github/Content/sources/baike'
os.chdir(path_Baike)

#%load_ext autoreload
%reload_ext autoreload
from Baike import Baike
baike = Baike()

In [ ]:
# Check for unmatched records and perform mannual matches
df_curr_releases['匹配片名'] = df_curr_releases['片名']
df_matched = df_reg[df_reg['匹配片名'].isin(df_curr_releases['片名'].to_numpy())]
idx_no_match = df_curr_releases.loc[~df_curr_releases['片名'].isin(df_matched['片名'].to_numpy())].index
df_curr_releases.loc[df_curr_releases.index.isin(idx_no_match), '匹配片名'] = None
df_curr_releases.loc[df_curr_releases['匹配片名'].isna(), :]

In [ ]:
# assign manually found titles to df_curr_releases
found = ['我们县里的年轻人', '最佳前女友',  '水击三千里', '草样年华',  '嫌疑人3', '草原英雄', 
 '随波逐流', '走了曹生',  '捣蛋鬼蜕变记', 
 '嘿·女朋友', '冰雪长津湖', '程序猿和鼓励狮']
df_curr_releases.loc[df_curr_releases['匹配片名'].isna(), '匹配片名'] = found

In [ ]:
df_reg.loc[df_reg['备案单位'].fillna('').str.contains('青岛星力量'), ['片名', '备案立项号', '备案单位', '编剧', '梗概', '公示日期']][:20]

In [ ]:
df_reg.loc[df_reg['备案立项号'].fillna('').str.contains('第3094号')]

In [ ]:
df_reg.loc[df_reg['编剧'].fillna('').str.contains('郑光春'), 
           ['片名', '备案单位', '编剧', '梗概', '备案立项号']] 

In [ ]:
data = baike.search('我的青春有个你')
baike.parse_content(data)

'我的青春有个你：我的青春有个你我的青春有个你：灵儿在教书著网络小说我的青春有个你：2021年王可如、刘冬沁主演的电影'

In [ ]:
pd.read_html(data)

In [ ]:
df_curr_releases.columns

Index(['电影类别', '编码', '公映证号', '片名', '第一出品单位', 'fid', '主演', '上映日期', '想看',
       '匹配片名'],
      dtype='object')

In [ ]:
df_matched.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '备案公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '电影类别', '匹配片名'],
      dtype='object')

In [ ]:
df_reg.columns

Index(['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期', '公示批次名称',
       '备案详细页链接', '公示批次链接', '电影类别', '匹配片名'],
      dtype='object')

In [ ]:
# Merge Manually Identified name
df_matched = df_reg.loc[df_reg['匹配片名'].isin(df_curr_releases['匹配片名'].to_numpy()),
                        ['匹配片名', '编剧', '梗概', '公示日期', 
                         '备案单位', '备案结果']
                       ]
df_matched.rename(columns={'公示日期':'备案公示日期'}, inplace=True)
df = df_curr_releases.merge(df_matched, on='匹配片名', how='left')
df['备案公示日期'] = df['备案公示日期'].apply(
    lambda x: pd.to_datetime(x, infer_datetime_format=True))
df['备案至今'] = dt_curr_release - df['备案公示日期']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype          
---  ------  --------------  -----          
 0   电影类别    34 non-null     object         
 1   编码      34 non-null     object         
 2   公映证号    34 non-null     object         
 3   片名      34 non-null     object         
 4   第一出品单位  34 non-null     object         
 5   fid     4 non-null      float64        
 6   主演      4 non-null      object         
 7   上映日期    4 non-null      object         
 8   想看      0 non-null      float64        
 9   匹配片名    34 non-null     object         
 10  编剧      34 non-null     object         
 11  梗概      34 non-null     object         
 12  备案公示日期  34 non-null     datetime64[ns] 
 13  备案单位    34 non-null     object         
 14  备案结果    34 non-null     object         
 15  备案至今    34 non-null     timedelta64[ns]
dtypes: datetime64[ns](1), float64(2), object(12), timedelta64[ns](1)
memory usage: 4.5

# [5] Run Models for prediction

## Predict Genre

In [ ]:
# Define Labels
label2id_genre= {'传奇': 0, '传记': 1, '其它': 2, '军旅': 3, '农村': 4, '宫廷': 5,  
                  '武打': 6, '涉案': 7, '神话': 8, '科幻': 9, '都市': 10, '青少': 11,
                  '革命': 12}
id2label_genre = {}
for k, v in label2id_genre.items(): id2label_genre[v] = k

In [ ]:
%%capture
# Genre Classifier
N_LABELS = len(id2label_genre)
ID2LABEL = id2label_genre
LABEL2ID = label2id_genre

from transformers import BertTokenizer, BertForSequenceClassification
import torch
############
model_name = '/content/drive/MyDrive/Github/Content/tools/models/chinese-bert-wwm-classification-NRTAgenre'
############
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
  model_name,
  num_labels=N_LABELS,
  problem_type="multi_label_classification",
  id2label=ID2LABEL,
  label2id=LABEL2ID,  
).to(device)
model.eval() # set model to eval mode for faster prediction

In [ ]:
batch_size = 8
i = 0
ls = df['梗概'].tolist()
softmax = torch.nn.Softmax(dim=-1)
L = df.shape[0]
test_predictions = []

while i < L:
  batch_test = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  batch_test.to(device)
  batch_outputs = model(**batch_test)
  batch_logtis = batch_outputs.logits
  batch_softmax = softmax(batch_logtis)
  batch_results = torch.argmax(batch_softmax, dim=1).cpu().numpy()
  test_predictions.extend(list(batch_results))
  i += batch_size

df['类型'] = test_predictions
df['类型'] = df['类型'].apply(lambda x: id2label_genre[x])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


## Predict Time

In [ ]:
# Define Labels
label2id_time = {'古代': 0, '当代': 1, '现代': 2, '近代': 3}
id2label_time = {}
for k, v in label2id_time.items(): id2label_time[v] = k

In [ ]:
%%capture
# Time Classifier

N_LABELS = len(id2label_time)
ID2LABEL = id2label_time
LABEL2ID = label2id_time

from transformers import BertTokenizer, BertForSequenceClassification
import torch
############
model_name = '/content/drive/MyDrive/Github/Content/tools/models/chinese-bert-wwm-classification-NRTAtime'
############
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
  model_name,
  num_labels=N_LABELS,
  problem_type="multi_label_classification",
  id2label=ID2LABEL,
  label2id=LABEL2ID,  
).to(device)
model.eval() # set model to eval mode for faster prediction

In [ ]:
batch_size = 8
i = 0
ls = df['梗概'].tolist()
softmax = torch.nn.Softmax(dim=-1)
L = df.shape[0]
test_predictions = []

while i < L:
  batch_test = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  batch_test.to(device)
  batch_outputs = model(**batch_test)
  batch_logtis = batch_outputs.logits
  batch_softmax = softmax(batch_logtis)
  batch_results = torch.argmax(batch_softmax, dim=1).cpu().numpy()
  test_predictions.extend(list(batch_results))
  i += batch_size

df['年代'] = test_predictions
df['年代'] = df['年代'].apply(lambda x: id2label_time[x])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


## Predict Title

In [ ]:
%%capture
from transformers import BertTokenizer, BartForConditionalGeneration

# assign device
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

# Instantiate tokenizer and model
checkpoint = "/content/drive/MyDrive/Github/Content/tools/models/PredTitle-10000"

  
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint,
                                            gradient_checkpointing=True, 
                                            use_cache=False)
model.to(device)
model.eval()

batch_size = 8
i = 0
ls = df['梗概'].tolist()
L = df['梗概'].shape[0]
test_predictions = []

while i < L:
  inputs = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  inputs.to(device)
  summary_ids = model.generate(input_ids=inputs['input_ids'],
                             num_beams=4,
                             min_length=0,
                             max_length=32
                             )
  
  ret = [tokenizer.decode(g, 
                         skip_specical_tokens=True, 
                         clean_up_tokenization_spaces=True) for g in summary_ids]
  test_predictions.extend(ret)
  i += batch_size

df['预测片名'] = test_predictions

def remove_specials(x):
  x = re.sub(' ', '', x)
  x = re.sub('\[CLS\]', '', x)
  x = re.sub('\[PAD\]', '', x)
  x = re.sub('\[SEP\]', '', x)
  return x

df['预测片名'] = df['预测片名'].apply(remove_specials)

## Extract top keywords

In [ ]:
import jieba.analyse
tfidf = jieba.analyse.extract_tags
textrank = jieba.analyse.textrank

df['kw'] = df['梗概'].apply(textrank, topK=10)

## Identify Main Characters

In [ ]:
import jieba
import jieba.posseg as pseg
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持

Paddle enabled successfully......


In [ ]:
# 识别主要角色
def find_PER(x: pd.Series):
  words = pseg.cut(x, use_paddle=True)
  ls_PER = []
  for word, flag in words:
    if flag == 'PER':
      word = word.strip('[\s,，。、“”？；]')
      ls_PER.append(word)
  ls_PER = list(set(ls_PER))

  ls_PER1 = ls_PER.copy()
  ls_PER = []
  while ls_PER1:
    PER =  ls_PER1.pop(0)
    ls_tmp = re.split('[\s,，。、“”？；]', PER)
    ls_PER.extend(ls_tmp)

  ls_PER = list(set(ls_PER)) 
  ls_PER = [PER for PER in ls_PER if len(PER) > 1]
  ls_PER_sorted = sorted(ls_PER, reverse=False, key=lambda x: len(x))

  ans = []
  while ls_PER_sorted:
    PER = ls_PER_sorted.pop(0)
    i = 0
    while i < len(ls_PER_sorted):
      if re.search(PER, ls_PER_sorted[i]):
        ls_PER_sorted.pop(i)
        ans.append(PER)
      i += 1
    if PER not in ans:
      ans.append(PER)

  return ans

In [ ]:
df['主要角色'] = df['梗概'].apply(find_PER)

## adjust genre

In [ ]:
df['修正类型'] = df['类型'].copy()
df['类型'].to_numpy()

array(['农村', '传奇', '传奇', '都市', '其它', '都市', '都市', '都市', '都市', '革命', '都市',
       '涉案', '青少', '青少', '都市', '涉案', '都市', '革命', '都市', '青少', '都市', '农村',
       '其它', '都市', '都市', '农村', '军旅', '其它', '都市', '其它', '都市', '都市', '其它',
       '都市'], dtype=object)

In [ ]:
df[['片名', '类型', '修正类型', '梗概']]

,片名,类型,修正类型,梗概
0,青春作伴好还乡,农村,农村,管松江带领以赵萌萌、李艳文为代表的一群年轻人在党的精准扶贫政策引领下，帮助洮西县脱贫致富，下...
1,孙悟空大战红孩儿（绍剧）,传奇,传奇,唐僧师徒西行取经，被八百里火焰山挡住去路。“火云洞”洞主红孩儿，凭着口吐“三昧神火”的小技，...
2,刺客道,传奇,传奇,明朝天启年间，出身卑微的殷影织遇到一个存在上千年的刺客组织“暗影会”，被告知自己身上流着刺客...
3,最佳射手,都市,青少,从小痴迷足球的林俊凯立志成为一名职业球员。争得父母同意后来到专业足校受训，因缺乏专业训练遭到...
4,光泽之夏,其它,都市,叛逆的地质大学学生林泽和性格古怪的父亲林光启生活在一起，时常发？？盾，相反却和身在美国的母亲...
5,虚拟的谎言,都市,都市,与徐泽相亲的对象各个另类，最后都成了他的前女友，后来徐泽与苏甜甜一见钟情。在水苼帮助下，徐泽...
6,世界唯一一个你,都市,都市,实习医生陈筱澜与音乐系学生穆阳一见倾心，成为一对甜蜜恋人。然而好景不长，陈筱澜突患癌症。不忍...
7,我们的岁月,都市,都市,该影片讲述了在那个特殊的时代，一群热血青年如何将他们的中国梦一步一步落在实处，如何在时代巨变...
8,绿皮小火车,都市,都市,通过一名大夫的视角，展现石嘴山市建市60来的光辉历程，从一个戈壁滩建成现在的工山水园林工业...
9,红麦,革命,革命,:中国工农红军二、六军团长征过丽江。电影《红麦》以此为背景，讲述了红军女战士与纳西族汉子共克...


## Save Records Curr Issue

In [ ]:
# Save records for current release issue
df.to_pickle(path_records + '/df_releases_{}.pkl'.format(str_date_curr_release))

In [ ]:
#df = pd.read_pickle(path_records + '/df_releases_{}.pkl'.format(str_date_curr_release))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype          
---  ------  --------------  -----          
 0   电影类别    34 non-null     object         
 1   编码      34 non-null     object         
 2   公映证号    34 non-null     object         
 3   片名      34 non-null     object         
 4   第一出品单位  34 non-null     object         
 5   fid     4 non-null      float64        
 6   主演      4 non-null      object         
 7   上映日期    4 non-null      object         
 8   想看      0 non-null      float64        
 9   匹配片名    34 non-null     object         
 10  编剧      34 non-null     object         
 11  梗概      34 non-null     object         
 12  备案公示日期  34 non-null     datetime64[ns] 
 13  备案单位    34 non-null     object         
 14  备案结果    34 non-null     object         
 15  备案至今    34 non-null     timedelta64[ns]
 16  类型      34 non-null     object         
 17  年代      34 non-null     object       

## View Table


In [ ]:
# Formulat view table
tmp = df[['备案单位','第一出品单位']].apply(
    lambda x: re.search(x['第一出品单位'], x['备案单位']), axis=1).to_frame()
idx_co_change = tmp[tmp[0].isna()].index
df['公司变更'] = 0
df.loc[idx_co_change, '公司变更'] = 1
df_view = df[['片名', '匹配片名','梗概', '备案至今', '备案结果', 
              '第一出品单位', '备案单位', '公司变更', '编剧', 
              '类型', '年代', '上映日期', '主演', '预测片名', 'kw',
              '主要角色', '修正类型'
              ]
            ].copy()
df_view.loc[df_view['片名'] == df_view['匹配片名'], '匹配片名'] = ''
df_view.rename(columns={'匹配片名':'备案名'}, inplace=True)
df_view.loc[df_view['公司变更'] == 0, '备案单位'] = ''
df_view.rename(columns={'备案单位':'原备案单位'}, inplace=True)
df_view.loc[df_view['备案结果'].str.contains('修改'),'备案结果'] = '是'
df_view.loc[df_view['备案结果'].str.contains('同意'),'备案结果'] = ''
df_view.rename(columns={'备案结果':'是否修改'}, inplace=True)
df_view['备案至今'] = round((df_view['备案至今'].dt.days + 60) / 365, 1)
df_view.sort_values(by='备案至今', ascending=False, inplace=True)
df_view['备案至今'] = df_view['备案至今'].apply(lambda x: '> ' + str(x) + ' 年')
df_view.drop('公司变更', axis=1, inplace=True)
df_view.loc[df_view['上映日期'].str.contains('今天').fillna(False),
            '上映日期'] = '上映中'

In [ ]:
df_view

# [6] Generate WordCloud

In [ ]:
import wordcloud
from PIL import Image
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
%matplotlib inline

path_font = '/content/drive/MyDrive/Github/Article/fonts/STHUPO.TTF'
path_img = '/content/drive/MyDrive/Github/Article/img'

def generate_word_image(ls_words, img_name):
  path_genre_icon = path_img + '/genre_icon/{}'.format(img_name)
  path_mask_image = path_img + '/genre_icon/mask_{}'.format(img_name)
  path_color_image = path_img + '/genre_icon/{}'.format(img_name)

  #wordcloud.color_from_image(coloring)
  mask = np.array(Image.open(path_mask_image)) # 定义词频背景
  coloring = np.array(Image.open(path_color_image)) # Load Image for coloring
  image_colors = wordcloud.ImageColorGenerator(coloring, default_color=(79, 46, 47))
  #image_colors = wordcloud.ImageColorGenerator(coloring)
  #wc.recolor(color_func=image_colors,  random_state=3)

  wc = wordcloud.WordCloud(
      font_path=path_font,
      width = 400,
      height = 100,
      scale = 1,
      mask=mask, # set back ground mask image
      max_words=188,
      max_font_size=88,
      min_font_size=2,
      #mode="RGBA",
      mode="RGB",
      background_color='white', 
      #background_color="rgba(255, 255, 255, 0)", 
      #contour_width=3, 
      #contour_color='steelblue',
      repeat=True,
      color_func=image_colors,
  )

  wfreq = Counter(ls_words)
  wc.generate_from_frequencies(wfreq)


  plt.imshow(wc) # 显示词云
  plt.axis('off') # 关闭坐标轴
  plt.show()

  fp_generated_img = (path_img + 
             '/ChinaFilm_NewReleases_Overview_'
             + ls_words[0] + '_' 
             + img_name.split('.')[0] + '_'
             + str_date_curr_release + '.png')
  wc.to_file(fp_generated_img)

  image = pil.open(fp_generated_img)
  image.thumbnail([300, sys.maxsize], pil.ANTIALIAS)
  image.save(fp_generated_img)


  return fp_generated_img

In [ ]:
id2label_time = {0: '古代', 1: '当代', 2: '现代', 3: '近代'}

id2label_genre = {0: '传奇', 1: '传记', 2: '其它', 3: '军旅', 4: '农村',
                  5: '宫廷', 6: '武打',  7: '涉案',   8: '神话',  9: '科幻',
                  10: '都市',  11: '青少',  12: '革命'
                 }
d_img_labels = {}
for t in id2label_time.values():
  for g in id2label_genre.values():
    d_img_labels[(t, g)] = None
d_img_labels.keys()

dict_keys([('古代', '传奇'), ('古代', '传记'), ('古代', '其它'), ('古代', '军旅'), ('古代', '农村'), ('古代', '宫廷'), ('古代', '武打'), ('古代', '涉案'), ('古代', '神话'), ('古代', '科幻'), ('古代', '都市'), ('古代', '青少'), ('古代', '革命'), ('当代', '传奇'), ('当代', '传记'), ('当代', '其它'), ('当代', '军旅'), ('当代', '农村'), ('当代', '宫廷'), ('当代', '武打'), ('当代', '涉案'), ('当代', '神话'), ('当代', '科幻'), ('当代', '都市'), ('当代', '青少'), ('当代', '革命'), ('现代', '传奇'), ('现代', '传记'), ('现代', '其它'), ('现代', '军旅'), ('现代', '农村'), ('现代', '宫廷'), ('现代', '武打'), ('现代', '涉案'), ('现代', '神话'), ('现代', '科幻'), ('现代', '都市'), ('现代', '青少'), ('现代', '革命'), ('近代', '传奇'), ('近代', '传记'), ('近代', '其它'), ('近代', '军旅'), ('近代', '农村'), ('近代', '宫廷'), ('近代', '武打'), ('近代', '涉案'), ('近代', '神话'), ('近代', '科幻'), ('近代', '都市'), ('近代', '青少'), ('近代', '革命')])

In [ ]:
df_label2image.loc[54, 'fn'] = 'tank-2466145_1920.png'
df_label2image.to_json(path_records + '/df_label2image.json')
df_label2image.tail(13)

,label,fn
52,传奇,fu-4075734_1920.png
53,传记,fighter-2732203_1920.png
54,军旅,tank-2466145_1920.png
55,农村,sun-5006687_1280.png
56,宫廷,temple-42519_1280.png
57,武打,fighter-2732203_1920.png
58,涉案,jail.png
59,神话,low-poly-2789706_1280.png
60,科幻,jupiter-55000.png
61,都市,cityurban.png


In [ ]:
from IPython.display import Image as Img
from PIL import Image as pil

path_posters = '/content/drive/MyDrive/Github/Article/img/posters'
path_icon = '/content/drive/MyDrive/Github/Article/img/genre_icon'

#df_label2image.to_json(path_records + '/df_label2image.json')
df_label2image = pd.read_json(path_records + '/df_label2image.json')

for i, row in df_label2image.tail(13).iterrows():
  image = pil.open(path_icon + '/' + row['fn'])
  #image.thumbnail([300, sys.maxsize], pil.ANTIALIAS)
  #image.save(path_posters + '/' + row['img_poster'], format='png')
  display(Img(path_icon + '/' + row['fn'], width=300))

In [ ]:
df.columns

Index(['电影类别', '编码', '公映证号', '片名', '第一出品单位', '备案名', '备案立项号', '备案单位', '编剧',
       '备案结果', '备案地', '梗概', '备案公示日期', '备案至今', 'time', 'genre', '类型', '年代',
       '预测片名', 'kw', '主要角色', '公司变更'],
      dtype='object')

In [ ]:
for i, f in df.drop_duplicates('片名'):
  ls = [f['片名']]*10
  ls += [f['预测片名']]*6
  ls += [f['主要角色']]*3
  if f['']

# [7] Generate Article

##[7.1] T1 标题：影片的前世今生

In [ ]:
# Title
T1 = '~{}月新片的前世今生~{}'.format(dt_curr_release.month, dt_curr_release.year)
T1

'~9月新片的前世今生~2021'

##[7.2] P1 2021年7月13日,又一批电影历经千辛万苦获得了公映许可证。本次

In [ ]:
df_view['上映日期']

In [ ]:
df_view[df_view['上映日期']== '上映中']

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期
14,我的青春有个你,草样年华,邱飞在大学生活中结识了一帮“不务正业”的理工男，他们整天琢磨怎样的人生才有意义……临近毕业时...,> 7.4 年,,上海爱美影视文化传媒有限公司,新丽传媒股份有限公司,孙睿,都市,当代,上映中
13,我的青春有个你,草样年华,大一入学伊始，男生邱飞踢球无意踢碎了女生周舟的暖壶，两人由此结识，开始了一场在大学的冒险之旅...,> 5.0 年,,上海爱美影视文化传媒有限公司,北京星汉影视文化传媒有限公司,孙睿,青少,当代,上映中
16,关于我妈的一切,,青岛姑娘小美在北京做记者，一心想逃离母亲季佩珍对她的掌控。母亲突然患癌的消息彻底颠覆了她的生...,> 1.4 年,,中国电影股份有限公司,上海喜焰文化发展有限公司,李洁、赵天宇,都市,当代,上映中


In [ ]:
P1 = ''
P1 +='{Y}年{M}月{D}日,'.format(Y=dt_curr_release.year, M=dt_curr_release.month, D=dt_curr_release.day)
P1 += '又一批电影历经千辛万苦获得了公映许可证。'

P1 += '本次获领证的影片共计{}部，'.format(df_curr_releases.shape[0])
P1 += '距离备案到获取公映证平均超过{}年。'.format(round(df_view['备案至今'].apply(lambda x: float(x.lstrip('> ').rstrip(' 年'))).mean(),1))
P1 += '其中，改过名字的有{}部，'.format(df_view.loc[df_view['备案名'] != '',:].shape[0])
P1 += '换过第一出品单位的有{}部，'.format(df.loc[df_view['原备案单位'] != '',:].shape[0])
P1 += '修改后才通过备案的有{}部，'.format(df.loc[df_view['是否修改'] == '是',:].shape[0])
if df_curr_releases.shape[0] - df_view.shape[0] > 0:
  P1 += '神秘到无迹可寻的有{}部，'.format(df_curr_releases.shape[0] - df_view.shape[0])
P1 += '题材排名前五的分别是{}'.format('、'.join(df_genre_sorted['类型'][:4]))
P1 += '和{}。'.format(df_genre_sorted['类型'][4:5].iloc[0])

P1 += '目前已在影院放映得有{}部，'.format(
    df_view[df_view['上映日期']== '上映中'].shape[0])
P1 += '分别是《{}》，'.format(
    '》、《'.join(df_view.loc[df_view['上映日期'] == '上映中', '片名']))

tbr = df_view[df_view['上映日期'] != '上映中'].dropna()
P1 += '已确认排期的有'
for i, f in tbr.iterrows():
  P1 += '{}上画的'.format(f['上映日期'])
  P1 += '《{}》'.format(f['片名'])
  P1 += '、'

P1 = P1.rstrip('、')
P1 += '，瓜熟蒂落实属不易。'


print(P1) 

2021年9月23日,又一批电影历经千辛万苦获得了公映许可证。本次获领证的影片共计33部，距离备案到获取公映证平均超过2.4年。其中，改过名字的有13部，换过第一出品单位的有13部，修改后才通过备案的有2部，题材排名前五的分别是都市、其它、农村、青少和传奇。目前已在影院放映得有3部，分别是《我的青春有个你》、《我的青春有个你》、《关于我妈的一切》，已确认排期的有2021-09-30上画的《长津湖》，瓜熟蒂落实属不易。


##[7.3] P2 怀胎最久的影片是《红船》,从

In [ ]:
df_view[df_view['片名'] == df_view.iloc[0]['片名']]

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期
14,我的青春有个你,草样年华,邱飞在大学生活中结识了一帮“不务正业”的理工男，他们整天琢磨怎样的人生才有意义……临近毕业时...,> 7.4 年,,上海爱美影视文化传媒有限公司,新丽传媒股份有限公司,孙睿,都市,当代,上映中
13,我的青春有个你,草样年华,大一入学伊始，男生邱飞踢球无意踢碎了女生周舟的暖壶，两人由此结识，开始了一场在大学的冒险之旅...,> 5.0 年,,上海爱美影视文化传媒有限公司,北京星汉影视文化传媒有限公司,孙睿,青少,当代,上映中


In [ ]:
P2 = ''
P2 += '怀胎最久的影片是《{}》,'.format(df_view.iloc[0]['片名'])
P2 += '从备案到获得公映证超过{}年，'.format(df_view.iloc[0]['备案至今'].lstrip('> ').rstrip(' 年'))
P2 += '名字从《{}》进化成了《{}》，'.format(df_view.iloc[0]['备案名'], df_view.iloc[0]['片名'])

record_longest = df_view[df_view['片名'] == df_view.iloc[0]['片名']]
P2 += '第一出品方历经了{}'.format('、'.join(record_longest['原备案单位']))
P2 += '，又回到{}。'.format(
    df_view.iloc[0]['原备案单位'], df_view.iloc[0]['第一出品单位'])
#ls_choices = ['美好', '胜利', '牛逼']
#P2 += '坚持就是{}。'.format(ls_choices[random.randint(0,len(ls_choices)-1)])

print(P2)

怀胎最久的影片是《我的青春有个你》,从备案到获得公映证超过7.4年，名字从《草样年华》进化成了《我的青春有个你》，第一出品方历经了新丽传媒股份有限公司、北京星汉影视文化传媒有限公司，又回到新丽传媒股份有限公司。


## [7.4] P3 同样坚守的还有...

In [ ]:
ls_name_3plus_year = df_view.loc[df_view['备案至今'].apply(
    lambda x: float(x.lstrip('> ').rstrip(' 年')))>3, '片名'].tolist()[1:]
ls_name_3plus_year = ['《'+ name + '》' for name in ls_name_3plus_year]
P3 = ""
P3 += '坚守了3年以上的影片还有{}。'.format('、'.join(ls_name_3plus_year))

print(P3)

坚守了3年以上的影片还有《世上只有妈妈好》、《我的青春有个你》、《孙悟空大战红孩儿（绍剧）》、《青花马》、《我们的岁月》、《程序猿与鼓励狮》、《单程车票》、《一江春水》、《徐大胆回乡记》。


## [7.5] P4 上市公司相关

In [ ]:
df_curr_releases['第一出品单位'].unique()

array(['长影集团有限责任公司', '绍兴市加东文化传播有限公司', '华影纪元影视文化（北京）有限公司', '山东最幕文化传媒有限公司',
       '广东多多影视传媒有限公司', '珠海山中木文化传播有限公司', '宁夏红影星影视城有限责任公司',
       '丽江天云影视文化传媒有限公司', '浙江晟喜华视文化传媒有限公司', '泰安凤凰文化旅游发展有限公司',
       '青岛红影时代影视传媒有限公司', '上海爱美影视文化传媒有限公司', '福建寒武纪影视文化发展有限公司',
       '中国电影股份有限公司', '拉萨净土数字经济产业集团有限公司', '龙传龙影视传媒（重庆）有限公司',
       '宁波市溪口雪窦影视传媒有限公司', '电影频道节目中心', '山东泰岳兄弟影视有限公司', '绍兴越剧艺术发展有限公司',
       '麦颂影视投资（上海）有限公司', '青岛众一嘉禾文化传媒有限公司', '北京博纳影业集团有限公司',
       '北京蒙太奇环球影业有限公司', '桂林瑞非永健文化传媒有限公司', '云南金灿影视文化传媒有限公司',
       '四川若城文化传媒有限公司', '峨眉电影集团有限公司', '青海大唐文化产业集团有限公司', '苏州花儿红电影制作有限公司'],
      dtype=object)

In [ ]:
# 本批次中与上市影视公司关联的项目有
info_public_film_co = pd.read_csv('/content/drive/MyDrive/Github/Article/reference/info_public_film_co.csv', index_col=0, encoding='utf-8-sig')
ls = info_public_film_co['公司简称'].apply(eval).sum()
pat_public = '|'.join(ls)
pat_public += '|阿里|腾讯|爱奇艺|英皇|寰亚|银都|美亚|大盛|儒意|灿星|横店|博纳|麦颂'
df_focus = df_view.loc[df_view['第一出品单位'].str.contains(pat_public, regex=True)]
df_focus = df_focus[df_focus['片名'] != '顺子加油']
df_focus

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期
26,长津湖,冰雪长津湖,1950年冬天，冰天雪地的长津湖区域，一场美国与中国炼狱般的血战即将拉开序幕。中美双方分别派...,> 2.1 年,,北京博纳影业集团有限公司,,贾燕江,军旅,现代,2021-09-30
16,关于我妈的一切,,青岛姑娘小美在北京做记者，一心想逃离母亲季佩珍对她的掌控。母亲突然患癌的消息彻底颠覆了她的生...,> 1.4 年,,中国电影股份有限公司,上海喜焰文化发展有限公司,李洁、赵天宇,都市,当代,上映中
23,只要你过得比我好,走了曹生,曹德望养大了妻子出轨生下的孩子曹生，可曹德望发现自己命不久矣，他希望能在死之前安顿好曹生，他...,> 1.1 年,,麦颂影视投资（上海）有限公司,北京冲击力影视投资有限公司,张翀,都市,当代,NaN


In [ ]:
idx_to_keep = [26, 16]
df_focus_narrowed = df_focus[df_focus.index.isin(idx_to_keep)]

In [ ]:
df_focus_narrowed

,片名,备案名,梗概,备案至今,是否修改,第一出品单位,原备案单位,编剧,类型,年代,上映日期
26,长津湖,冰雪长津湖,1950年冬天，冰天雪地的长津湖区域，一场美国与中国炼狱般的血战即将拉开序幕。中美双方分别派...,> 2.1 年,,北京博纳影业集团有限公司,,贾燕江,军旅,现代,2021-09-30
16,关于我妈的一切,,青岛姑娘小美在北京做记者，一心想逃离母亲季佩珍对她的掌控。母亲突然患癌的消息彻底颠覆了她的生...,> 1.4 年,,中国电影股份有限公司,上海喜焰文化发展有限公司,李洁、赵天宇,都市,当代,上映中


In [ ]:
P4 = ''
P4 += '本批次中，大数据分析识别出{}部上市影视公司关联项目，'.format(df_focus.shape[0])
P4 += '占比{}%。'.format(round( (df_focus.shape[0]/(df_view.shape[0]-1)*100), 2 ) )
P4 += '结合题材与出品方实力，慷田AI聚焦关注《{}》'.format(
    '》、《'.join(df_focus_narrowed['片名']))
P4 += '，题材类型包括了{}。'.format(
    '、'.join(df_focus_narrowed.类型.unique()))

print(P4)

本批次中，大数据分析识别出3部上市影视公司关联项目，占比9.09%。结合题材与出品方实力，慷田AI聚焦关注《长津湖》、《关于我妈的一切》，题材类型包括了军旅、都市。


##[7.6] P5 按影片展示词云图

In [ ]:
df_label2image

,label,fn
0,"[古代, 传奇]",None
1,"[古代, 传记]",vintage-5323524_1920
2,"[古代, 其它]",lampion-1509664_1920
3,"[古代, 军旅]",None
4,"[古代, 农村]",None
...,...,...
60,科幻,jupiter-55000.png
61,都市,cityurban.png
62,青少,art-2687649_1280.png
63,革命,muscle-1085672_1280.png


In [ ]:
%%capture
df_view['src_img'] = None
df_view['tgt_img'] = None

In [ ]:
genre_count = df_view.groupby('修正类型')['片名'].count(
    ).rename('类型数量').reset_index()
df_view = df_view.merge(genre_count, on='修正类型', how='left')
df_view = df_view.sort_values(['类型数量','上映日期'], ascending=[False, True])
df_view

In [ ]:
P5 = ''
P5 = '\n看图快速了解本期影片。'

print(P5)


看图快速了解本期影片。


In [ ]:
idx_actor = df_view['主演'].dropna().index

for i in df_view.drop_duplicates('片名').index:
  ls = [df_view.loc[i, '片名']] * 20
  ls += [df_view.loc[i, '预测片名']] *6
  ls += df_view.loc[i, '主要角色']*3
  if i in idx_actor:
    ls += df_view.loc[i, '主演']*3
  ls += [df_view.loc[i, '修正类型']] *3
  ls += [df_view.loc[i, '年代']]*2
  ls += df_view.loc[i, 'kw']
  ls += [df_view.loc[i, '第一出品单位']]
  img_fn = df_view.loc[i, 'src_img']
  if not img_fn:
    img_fn = df_label2image.loc[
        df_label2image['label'] == df_view.loc[i, '修正类型'],
        'fn'
    ].iloc[0]
  txt = '《{}》'.format(df_view.loc[i, '片名'])
  if not pd.isna(df_view.loc[i, '上映日期']):
    if df_view.loc[i, '上映日期'] == '上映中':
      txt += "({})".format(df_view.loc[i, '上映日期'])
    else:
      txt += "({}上映)".format(df_view.loc[i, '上映日期'])
  txt += '，备案至今{}'.format(df_view.loc[i, '备案至今'])

  if df_view.loc[i, '备案名'] != '':
    txt += '\n原备案片名：《{}》'.format(df_view.loc[i, '备案名'])

  txt += '\n编剧：{}'.format(df_view.loc[i, '编剧'])
  #txt += '\n类型：{}'.format(df_view.loc[i, '类型'])

  txt += '\n第一出品单位：{}'.format(df_view.loc[i, '第一出品单位'])
  if df_view.loc[i, '原备案单位'] != '':
    txt +=  '\n原备案单位：{}'.format(df_view.loc[i, '原备案单位'])
  
  print(txt)
  #txt += '{}\n'.format(df_view.loc[i, '梗概'])
  fp_generated_img = generate_word_image(ls, img_fn)
  display(fp_generated_img)
  df_view.loc[i, 'tgt_img'] = fp_generated_img

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
P5 = ''
P5 = '看图了解这期推出的电影都讲了什么。'

print(P5)

看图了解这期推出的电影都讲了什么。


## [7.7] 指向电影官网

In [ ]:
#####
E1 = '\n\n'
E1 += ''
E1 += '慷田AI结合自主调研及多方大数据比对，通过分析、建模，提炼关键信息。'
E1 += '公映许可证公示信息来自中国国家电影局 China Film Administration, 官方网址 '
E1 += ' http://www.chinafilm.gov.cn/chinafilm 。\n'

print(E1)



慷田AI结合自主调研及多方大数据比对，通过分析、建模，提炼关键信息。公映许可证公示信息来自中国国家电影局 China Film Administration, 官方网址  http://www.chinafilm.gov.cn/chinafilm 。



In [ ]:
 df_view['片名'].to_numpy()[0:5]

array(['我的青春有个你', '世上只有妈妈好', '我的青春有个你', '孙悟空大战红孩儿（绍剧）', '青花马'],
      dtype=object)

## [8.9] E2\E3, Pointer Text

In [ ]:
E2 = '点击阅读原文查看本期慷田AI影片信息详表。'
E3 = '点击阅读原文查看本期慷田AI影片概览分析。'

## [7.8] S0 生成摘要

In [ ]:
S0 = '{}月新片{}部，'.format(dt_curr_release.month, df_curr_releases.shape[0])
S0 += '平均孕期超{}年，'.format(round(df_view['备案至今'].apply(lambda x: float(x.lstrip('> ').rstrip(' 年'))).mean(),1))
S0 += '改名的{}部，'.format(df.loc[df_view['备案名'] != '',:].shape[0])
S0 += '换主的{}部，'.format(df.loc[df_view['原备案单位'] != '',:].shape[0])
S0 += '题材集中在{}'.format('、'.join(df_genre_sorted['类型'][:4]))
S0 += '和{}。'.format(df_genre_sorted['类型'][4:5].iloc[0])
S0 += '慷田AI聚焦'
focus = ['《'+ name + '》' for name in df_focus_narrowed['片名'].to_numpy()]
S0 += '{}'.format('、'.join(focus))
S0 += '…'

print(S0)

9月新片33部，平均孕期超2.4年，改名的13部，换主的13部，题材集中在都市、其它、农村、青少和传奇。慷田AI聚焦《长津湖》、《关于我妈的一切》…


# [5] Output Word Doc

## [5.1] Install Docx Lib

In [ ]:
%%capture
!pip install python-docx
!pip install lxml
from docx import Document
from docx.shared import Inches
from docx.oxml.ns import qn
#from docx.text.parargaph import Paragraph

from docx.shared import Pt

## [5.2] Write Overview to docx and download

In [ ]:
df_view['备案至今']

In [ ]:
path_doc = '/content/drive/MyDrive/Github/Article/docx'
path_img = '/content/drive/MyDrive/Github/Article/img'

doc = Document()
doc.core_properties.title = 'ChinaFilm_NewReleases_Overview_' + str_date_curr_release
style = doc.styles['Normal']
font = style.font
font.name = 'simhei'
font.size = Pt(12)

doc.add_heading(T1, 0)

p = doc.add_paragraph(S0, style='Intense Quote')

doc.add_paragraph(P1)

doc.add_picture(path_img + '/posters/' + '长津湖.png', width =Inches(6))

doc.add_paragraph(P2)

doc.add_paragraph(P3)

doc.add_picture(path_img + '/posters/' + '关于我妈的一切.png', width =Inches(6))

doc.add_paragraph(P4)

doc.add_paragraph(P5)
for i in df_view.drop_duplicates('片名').index:
  txt = '\n《{}》'.format(df_view.loc[i, '片名'])
  if not pd.isna(df_view.loc[i, '上映日期']):
    if df_view.loc[i, '上映日期'] == '上映中':
      txt += "({})".format(df_view.loc[i, '上映日期'])
    else:
      txt += "({}上映)".format(df_view.loc[i, '上映日期'])
  txt += '，备案至今{}'.format(df_view.loc[i, '备案至今'])
  if df_view.loc[i, '备案名'] != '':
    txt += '\n原备案片名：《{}》'.format(df_view.loc[i, '备案名'])

  txt += '\n编剧：{}'.format(df_view.loc[i, '编剧'])
  #txt += '\n类型：{}'.format(df_view.loc[i, '类型'])

  txt += '\n第一出品单位：{}'.format(df_view.loc[i, '第一出品单位'])
  if df_view.loc[i, '原备案单位'] != '':
    txt +=  '\n原备案单位：{}'.format(df_view.loc[i, '原备案单位'])

  fp_img = df_view.loc[i, 'tgt_img']
  doc.add_paragraph(txt)
  doc.add_picture(fp_img, width =Inches(4))


doc.add_paragraph(E1)
doc.add_paragraph(E2)

doc.save(path_doc + '/ChinaFilm_NewReleases_Overview_' + str_date_curr_release +'.docx')

In [ ]:
# Download Document
from google.colab import files

files.download(
    path_doc + '/ChinaFilm_NewReleases_Overview_' + str_date_curr_release +'.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## [5.3] Write Table to docx and download

In [ ]:
print(T1)

~9月新片的前世今生~2021


In [ ]:
path_doc = '/content/drive/MyDrive/Github/Content/tools/articles/docx'

doc = Document()
doc.core_properties.title = 'ChinaFilm_NewReleases_Table_' + str_date_curr_release

doc.add_heading('详表：' + T1, 0)

p = doc.add_paragraph(S0, style='Intense Quote')

table = doc.add_table(rows=1, cols=1, style='Light List Accent 4')
i = 1
for i, row in df_view.iterrows():
  row_cells = table.add_row().cells
  row_cells[0].text = '[' + str(i) + ']' + '《'.format(i) + row['片名'] + '》'
  if row['备案名'] != '':
    row_cells[0].text += ' (原备案名《{}》)'.format(row['备案名'])

  row_cells = table.add_row().cells
  row_cells[0].text = '编剧: ' + row['编剧'] 
  
  row_cells = table.add_row().cells
  row_cells[0].text = '距离备案 ' + row['备案至今']
  if row['是否修改'] == '是':
    row_cells[0].text += ', 修改后通过备案'

  row_cells = table.add_row().cells
  row_cells[0].text = '第一出品单位: ' + row['第一出品单位']
  if row['原备案单位'] != '': 
    row_cells = table.add_row().cells
    row_cells[0].text = '原备案单位：{}'.format(row['原备案单位'])
  
  row_cells = table.add_row().cells
  row_cells[0].text = '类型: ' + row['修正类型'] 

  idx_actor = df_view['主演'].dropna().index
  if i in idx_actor:
    row_cells = table.add_row().cells
    row_cells[0].text = '主演: ' + '、'.join(row['主演']) 

  row_cells = table.add_row().cells
  row_cells[0].text = row['梗概'].lstrip('\w+')

  row_cells = table.add_row().cells
  row_cells[0].text = ''
  i+=1

doc.add_paragraph(E1)
doc.add_paragraph(E3)

doc.save(path_doc + '/ChinaFilm_NewReleases_Table_' +
         str_date_curr_release +'.docx')

In [ ]:
files.download(path_doc + '/ChinaFilm_NewReleases_Table_' +
         str_date_curr_release +'.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>